In [1]:
from uk_covid19 import Cov19API
import json
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [2]:
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

In [3]:
def normaliseDate(dat):
    ##wrangles data and makes the date the index
    dat = dat.get("data")
    df = pd.json_normalize(dat)
    df['date'] = pd.to_datetime(df["date"])
    df = df.loc[(df['date'] >= '2020-05-01') & (df['date'] < '2022-05-01')]
    df.set_index("date", inplace = True)
    return df

def normaliseAge(field, df):
    ##wrangles data and makes age the index
    df = df.get("data")[0]
    df = pd.json_normalize(df.get(field))
    df = df.set_index("age")
    df = df.drop(['value'], axis = 1) #raw numbers not too helpful here due to population pyramid, so I'm discarding
    return df

def newColumns(df): 
    ## adding case fatality & case hospitalisation
    df['case-21'] = df['cases'].shift(-21) #accounts for ~3 week latency between infection & death?
    df['case-14'] = df['cases'].shift(-14) #accounts for ~2 week incubation prior to hospitalisation?
    df['caseFatality'] = np.where(df['case-21'] != 0, df['deathRate']/df['case-21']*100, np.nan)
    df['caseHospitalisation'] = np.where(df['case-14'] != 0, df['hospitalRate']/df['case-21']*100, np.nan)
    df = df.drop(['case-21','cases','case-14','deathRate','hospitalRate'], axis = 1) #we don't need these anymore
    return df

def access_api(button):
    apibutton.icon="check"
    apibutton.disabled=True

In [4]:
with open("deaths.json","rt") as INFILE:
    deaths=json.load(INFILE)
with open("EngOverview.json","rt") as INFILE:
    EngOverview = json.load(INFILE)
with open("ScoOverview.json","rt") as INFILE:
    ScoOverview = json.load(INFILE)

In [5]:
##producing dataframes
maleDeaths = normaliseAge("maleDeaths", deaths)
femaleDeaths = normaliseAge("femaleDeaths", deaths)
allDeaths_df = pd.merge(maleDeaths, femaleDeaths, on='age', suffixes=('_male', '_female'))
allDeaths_df.rename(columns={'rate_male':'Male Deaths', 'rate_female':'Female Deaths'}, inplace=True)
allDeaths_df.to_pickle("allDeaths_df.pkl")

In [6]:
##producing dataframes
Eng_df = newColumns(normaliseDate(EngOverview))
Sco_df = newColumns(normaliseDate(ScoOverview))
Eng_df.to_pickle("EngProg_df.pkl")
Sco_df.to_pickle("ScoProg_df.pkl")

In [7]:
##incorporating widget selections
def updatePlotPercent(series,scale):
    if series == 'England':
        Eng_df.plot(title='England Vaccination vs. COVID-19 Prognosis', color = ['blue','red','orange'])
    elif series == 'Scotland':
        Sco_df.plot(title='Scotland Vaccination vs. COVID-19 Prognosis', color = ['blue','red','orange'])
    plt.xlabel('Date')
    plt.ylabel('Percent')
    if scale == 'log':
        plt.yscale('log')
        plt.ylim(0.1, 100)
    else:
        plt.ylim(0, 100) #Standardising y-axis for ease of comparison
    plt.show()
    
def updateBar(sex):
    plt.figure(figsize=(10, 6))
    if sex == 'Both':
        allDeaths_df.plot(kind='bar', title='Death Rate by Sex and Age')
    else:
        allDeaths_df[[f'{sex} Deaths']].plot(kind='bar', title=f'{sex} Death Rate by Age')
    plt.xlabel('Age')
    plt.ylabel('Death Rate')
    plt.ylim(0, 12000) #Standardising y-axis for ease of comparison
    plt.show()

In [8]:
Eng_df = pd.read_pickle("EngProg_df.pkl")
Sco_df = pd.read_pickle("ScoProg_df.pkl")
allDeaths_df = pd.read_pickle("allDeaths_df.pkl")

In [9]:
scaleRadio = wdg.RadioButtons(
    options=['linear', 'log'],
    disabled=False)
countrySelect = wdg.ToggleButtons(
    options=['England', 'Scotland'],
    description='Select Nation:',
    disabled=False
)

# Refresh Data

In [10]:
apibutton.on_click(access_api)
display(apibutton)

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

# Vaccination Effort in 2021

As you may recall the first year of the pandemic, the race to develop and administer a vaccine for SARS-CoV-2 among the general population was a major feature of public discourse. Clinical findings for most vaccines were highly encouraging, showing a high degree of protection against acute symptoms of the disease. In this figure, we have used the rolling death and hospitalisation rates for COVID-19 expressed as a proportion of positive tests (with specimen dates offset by 21 and 14 days, respectively, to account for the virus' incubation period) to serve as a proxy for case fatality and hopitalisation rates.

This is an admittedly imperfect proxy, partiuclarly as time passes and adherence to COVID-19 testing protocols declined. Even so, a rapid improvmement in the prognosis of positive tests can be observed that after a remarkably quick mass-vaccination programme at the start of 2021. Even accounting for other factors such as the rate of ambient natural immunity after widespread circulation of the virus, it can be reasonably concluded that the vaccination programme conferred durable degree of protection against severe disease on the population level.

In [11]:
wdg.interactive(updatePlotPercent,series = countrySelect, scale=scaleRadio)

interactive(children=(ToggleButtons(description='Select Nation:', options=('England', 'Scotland'), value='Engl…

# Death Rates by Age and Sex

It is well understood that COVID-19 mortality is concentrated disproportionately among the elderly. In the following graphic, we can see the degree to which this is the case, with elderly patients suffering a particularly high death rate. A sex-dependent effect is also observable, as death rates appear noticeably higher among men than women. 

In [12]:
options = ['Male', 'Female', 'Both']
dropdown = wdg.Dropdown(options=options, description='Sex:')
wdg.interactive(updateBar, sex=dropdown)

interactive(children=(Dropdown(description='Sex:', options=('Male', 'Female', 'Both'), value='Male'), Output()…